Data Ingestion Stage

In [11]:
from collections import namedtuple
import os
import sys
from dataclasses import dataclass
from pathlib import Path

In [12]:
os.chdir('/Users/praveenhome/Desktop/PRAVEENBASE/AWSDEVOPS/AWSDEVOPS2023/cnnClassifier/src/')

In [13]:
DataIngestionConfig = namedtuple("DataIngestionConfig", [
    "root_dir",
    "source_URL",
    "local_data_file",
    "unzip_dir"
])

In [15]:

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [16]:
print('Original sys.path:', sys.path)

Original sys.path: ['/Users/praveenhome/Desktop/PRAVEENBASE/AWSDEVOPS/AWSDEVOPS2023/cnnClassifier/research', '/Users/praveenhome/miniconda3/envs/cnncls/lib/python38.zip', '/Users/praveenhome/miniconda3/envs/cnncls/lib/python3.8', '/Users/praveenhome/miniconda3/envs/cnncls/lib/python3.8/lib-dynload', '', '/Users/praveenhome/miniconda3/envs/cnncls/lib/python3.8/site-packages']


In [17]:
sys.path.append("Users/praveenhome/Desktop/PRAVEENBASE/AWSDEVOPS/AWSDEVOPS2023/cnnClassifier/src/")

In [18]:
from cnnClassifier import logger
from cnnClassifier.constants import *
from cnnClassifier.utils import read_yaml , create_directories
from pathlib import Path

# CONFIG_FILE_PATH = Path("/config/config.yaml")
CONFIG_FILE_PATH = Path("/Users/praveenhome/Desktop/PRAVEENBASE/AWSDEVOPS/AWSDEVOPS2023/cnnClassifier/config/config.yaml")
PARAMS_FILE_PATH = Path("/Users/praveenhome/Desktop/PRAVEENBASE/AWSDEVOPS/AWSDEVOPS2023/cnnClassifier/params.yaml")


In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [20]:
import os
import urllib.request as request
from zipfile import ZipFile

In [21]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

    
    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpg") and ("Cat" in f or "Dog" in f)]
    
    
    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)

    

    
    def unzip_and_clean(self):
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)

In [22]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2023-12-27 20:11:23,460: INFO: common: yaml file: /Users/praveenhome/Desktop/PRAVEENBASE/AWSDEVOPS/AWSDEVOPS2023/cnnClassifier/config/config.yaml loaded successfully]
[2023-12-27 20:11:23,464: INFO: common: yaml file: /Users/praveenhome/Desktop/PRAVEENBASE/AWSDEVOPS/AWSDEVOPS2023/cnnClassifier/params.yaml loaded successfully]
[2023-12-27 20:11:23,465: INFO: common: created directory at: artifacts]
[2023-12-27 20:11:23,466: INFO: common: created directory at: artifacts/data_ingestion]


In [8]:
pip install ensure

  Using cached ensure-1.0.4-py3-none-any.whl.metadata (10 kB)
Using cached ensure-1.0.4-py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from ensure import ensure_annotations